In [18]:
import os
import sys

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm

from sklearn.cluster import KMeans, spectral_clustering
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import silhouette_score, silhouette_samples
from copy import deepcopy

In [19]:
root_path = "/home/g.esposito/sc2-benchmark"
layer_confs = [file_path for file_path in os.listdir(root_path) if file_path.startswith("FSIM_W_hpc_sem_L")]

detailed_report = pd.DataFrame([])
general_report = pd.DataFrame([])

for layer_conf in layer_confs:
    configurations_log = os.path.join(root_path, layer_conf)

    # to_iterate = [log for log in os.listdir(configurations_log) if os.path.isdir(configurations_log)]
    # print(to_iterate)
    for cnf in os.listdir(configurations_log):
        cnf_path = os.path.join(configurations_log, cnf) 
        if os.path.isdir(cnf_path):
            # print(os.path.isdir(configurations_log))
            # print(cnf)
            for file in os.listdir(cnf_path):
                report_path = os.path.join(cnf_path, file)
                # print(report_path)
                if file == "Faulty_masks_report.csv":
                    report = pd.read_csv(report_path)

                    report['conf_detail'] = cnf.split("_")[0] if cnf.split("_")[0] != "cnf77" else "teacher_cnf"
                    report['bit_faulty_position'] = np.log2(report['bitmask'])

                    report.drop(columns=['Unnamed: 0'], inplace=True)
                    detailed_report = pd.concat([detailed_report, report], axis=0, ignore_index=True)

                if file == "fsim_full_report.csv":
                    report = pd.read_csv(report_path)

                    report['conf_detail'] = cnf.split("_")[0] if cnf.split("_")[0] != "cnf77" else "teacher_cnf"

                    report['bit_faulty_position'] = np.log2(report['bitmask'])

                    indices = report.query("mask_Crit==0 and mask_SDC==0 and mask_Masked == 0").index
                    for idx in indices:
                        report.loc[idx, 'Crit_perc'] = 100
                        report.loc[idx, 'SDC_perc'] = 0
                        report.loc[idx, 'masked_perc'] = 100

                    report.drop(columns=['kernel.1','channel.1','row.1','col.1','BitMask', 'Unnamed: 0'], inplace=True)
                    # print(report.columns)

                    tot_mask_per_fault = report['mask_Crit'] + report['mask_SDC'] + report['mask_Masked']
                    # for val in tot_mask_per_fault:
                    #     if val == 0:
                    #         print('queste sono tutte critiche')
                    report['Crit_perc'] = (report['mask_Crit'] / tot_mask_per_fault)*100
                    report['SDC_perc'] = (report['mask_SDC'] / tot_mask_per_fault)*100
                    report['masked_perc'] = (report['mask_Masked'] / tot_mask_per_fault)*100

                    report['global_fault_f1@1'] = report['global_fault_f1@1']*100

                    general_report = pd.concat([general_report, report], axis=0, ignore_index=True)
                

detailed_report.columns
general_report.columns

Index(['layer', 'kernel', 'channel', 'row', 'col', 'bitmask',
       'global_gold_acc', 'global_gold_f1@1', 'global_fault_acc',
       'global_fault_f1@1', 'mask_Crit', 'mask_SDC', 'mask_Masked', 'Layer',
       'Ffree_Weight', 'Faulty_weight', 'Abs_error', 'conf_detail',
       'bit_faulty_position', 'Crit_perc', 'SDC_perc', 'masked_perc'],
      dtype='object')

In [20]:
detailed_report.isna().sum()

FaultID                     0
imID                        0
layer                       0
kernel                      0
channel                     0
row                         0
col                         0
bitmask                     0
label_idx                   0
f_class_iou            185687
f_label_acc            185687
f_label_area           359538
f_label_f1             369078
g_class_iou            410923
g_label_acc            410923
g_label_area           596610
g_label_f1             611927
iou_per_img                 0
conf_detail                 0
bit_faulty_position         0
dtype: int64

In [35]:
len(detailed_report)

4062614

In [34]:
print(f'Percentage of critical boxes because of the low iou score: {len(detailed_report.query("f_label_acc < 90 and not g_label_acc.isnull()"))/len(detailed_report)}')

Percentage of critical boxes because of the low iou score: 0.07882535726997446
